In [4]:
!python -m spacy download es_core_news_md -q

[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [5]:
import re
import spacy
import nltk
import torch
import pandas as pd
import numpy as np
from torch import nn
from nltk.corpus  import stopwords
from spellchecker import SpellChecker
from sklearn.preprocessing import LabelEncoder
from transformers import BertForSequenceClassification, BertTokenizer, BertModel

nltk.download('stopwords')
nlp = spacy.load("es_core_news_md")
stop_words = set(stopwords.words('spanish'))
spell = SpellChecker(language='es')

def minusculas(text):
    # Convertir a minúsculas
    text = text.lower()
    return text

def lematizar(texto):
    doc = nlp(texto)
    lemas = ' '.join([token.lemma_ for token in doc])
    return lemas

def signos_puntuacion(text):
    # Sustituir acentos por letras sin acentos
    text = re.sub(r'[áäâà]', 'a', text)
    text = re.sub(r'[éêèë]', 'e', text)
    text = re.sub(r'[íîìï]', 'i', text)
    text = re.sub(r'[óôòö]', 'o', text)
    text = re.sub(r'[úûùü]', 'u', text)
    # Eliminar signos de puntuación
    text = re.sub(r'[^\w\s]', '', text)
    return text

def remove_stop_words(text):
    words = text.split()
    filtered_text = []
    for word in words:
        if word.lower() not in stop_words:
            filtered_text.append(word)
    return ' '.join(filtered_text)

# Función para corregir el texto
def corregir_texto(texto):
    palabras = texto.split()
    #corregido = [spell.correction(palabra) for palabra in palabras]
    corregido = [spell.correction(palabra) if spell.correction(palabra) is not None else palabra for palabra in palabras]
    texto_corregido = ' '.join(corregido)
    return texto_corregido

# Definir la clase FineTuningBERT
class FineTuningBERT(nn.Module):
    def __init__(self, bert):
        super(FineTuningBERT, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(bert.config.hidden_size, 64)
        self.ReLU = nn.ReLU()
        self.fc2 = nn.Linear(64, 93)

        # Agregar la capa faltante para position_ids
        self.position_ids = nn.Embedding(bert.config.max_position_embeddings, bert.config.hidden_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_hs = outputs['pooler_output']

        x = self.dropout(cls_hs)
        x = self.fc1(x)
        x = self.ReLU(x)
        x = self.fc2(x)
        return x

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
ruta_modelo = 'modelo-clasficacion-preguntas-planetas.pth'
ruta_tokenizador = 'tokenizer-clasficacion-preguntas-planetas'

# Inicializar el modelo BERT pre-entrenado
bert_model = BertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", return_dict=True)

# Inicializar el modelo FineTuningBERT
model = FineTuningBERT(bert_model)
# Cargar los pesos del modelo guardados
model.load_state_dict(torch.load(ruta_modelo), strict=False)  # strict=False para omitir las claves faltantes
# Cargar el tokenizer desde su directorio
tokenizer = BertTokenizer.from_pretrained(ruta_tokenizador)


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.we

In [7]:
df = pd.read_csv("datosChat.csv")
# Creamos un objeto LabelEncoder
label_encoder = LabelEncoder()
# Convertimos las etiquetas de texto a números enteros
df['label_codigo'] = label_encoder.fit_transform(df['label'])
# Obtenemos la correspondencia entre números y etiquetas
label_dict = {i: label for i, label in enumerate(label_encoder.classes_)}

In [13]:
# Función que encapsula el procesamiento y la predicción
def procesar_y_predecir(texto_ejemplo, tokenizer = tokenizer, model=model , label_dict = label_dict):
    texto_procesado = minusculas(texto_ejemplo)
    texto_procesado = signos_puntuacion(texto_procesado)
    texto_procesado = corregir_texto(texto_procesado)
    texto_procesado = lematizar(texto_procesado)
    texto_procesado = remove_stop_words(texto_procesado)

    inputs = tokenizer(texto_procesado, return_tensors="pt", padding=True, truncation=True, max_length=128)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        probabilities = torch.softmax(outputs, dim=1)

    predicted_label_id = torch.argmax(outputs, dim=1).item()
    predicted_probability = probabilities[0][predicted_label_id].item()

    print("Texto de ejemplo original:", texto_ejemplo)
    print("Texto de ejemplo procesado:", texto_procesado)
    print("Clase predicha:", label_dict[predicted_label_id])
    #print(f"Score: {predicted_probability:.2f}")
    
    if predicted_probability >= .06:
        
        all_probabilities = probabilities[0].tolist()
        max_probability = max(all_probabilities)
        sorted_indexes = sorted(range(len(all_probabilities)), key=lambda k: all_probabilities[k], reverse=True)
    
        top_n = 5
        for i, idx in enumerate(sorted_indexes[:top_n], 1):
            probability = all_probabilities[idx]
            percentage = (probability / max_probability) * 100 if max_probability > 0 else 0
            #print(f"score {probability:.2f} de la opcion {i}, porcentaje: {percentage:.0f}%, Índice de etiqueta: {idx}, Etiqueta: {label_dict[idx]}")
        # Filtrar el DataFrame para obtener las filas con la etiqueta predicha
        filas_etiqueta_predicha = df[df['label_codigo'] == predicted_label_id]
        # Seleccionar aleatoriamente una fila
        fila_aleatoria = filas_etiqueta_predicha.sample(n=1)
        # Obtener el valor de la columna 'respuesta' de la fila seleccionada
        respuesta = fila_aleatoria['respuesta'].values[0]
        #print("Respuesta correspondiente a la etiqueta predicha:")
    else:
        respuesta = "Por favor, vuelva a hacer la solicitud."
    #print(respuesta)
    return respuesta

In [14]:
import speech_recognition as sr
import ipywidgets as widgets
from IPython.display import display, clear_output
import pyttsx3

# Inicializar el reconocedor de voz
recognizer = sr.Recognizer()

# Configuración del motor de texto a voz
def convert_text_to_audio(text):
    engine = pyttsx3.init()
    engine.setProperty('rate', 170) 
    engine.setProperty('voice', 'spanish')
    engine.say(text)
    engine.runAndWait()

# Función para procesar la solicitud de voz
def process_voice_request(button):
    with output:
        clear_output()
        if button.description == 'Iniciar conversacion':
            button.description = 'Procesando solicitud de voz'
            # Activar el reconocimiento de voz
            with sr.Microphone() as source:
                print("Te escucho...")
                recognizer.adjust_for_ambient_noise(source)
                #audio = recognizer.listen(source)
                audio = recognizer.listen(source, timeout=2)
            try:
                # Reconocer el audio y mostrar la respuesta
                text = recognizer.recognize_google(audio, language='es-ES')
                output_text = widgets.Label("Has dicho: " + text.lower())
                display(output_text)
                # Procesar la solicitud
                respuesta = procesar_y_predecir(text)
                output_resp = widgets.Textarea(value=respuesta, disabled=False, layout={'height': 'auto', 'width': '50%'}, rows=3)
                #output_resp = widgets.Label("Respuesta: " + respuesta.lower())
                # Establecer encabezado
                output_resp_label = widgets.Label(value="Respuesta:")
                #display(output_resp)
                display(output_resp_label, output_resp)
                # Convertir texto a audio y reproducirlo
                convert_text_to_audio(respuesta)
                button.description = 'Nueva solicitud'
            except sr.UnknownValueError:
                print("No se ha entendido lo que has dicho, vuelve a intentarlo...")
        else:
            button.description = 'Iniciar conversacion'
            clear_output()

In [15]:
# Crear el botón y el output
button = widgets.Button(description='Iniciar conversacion', layout=widgets.Layout(width='250px'), button_style='info')
output = widgets.Output()
# Conectar la función con el evento "on_click" del botón
button.on_click(process_voice_request)
# Conectar la función con el evento "on_click" del botón
button.on_click(process_voice_request)
# Mostrar la imagen y el botón
image_path = 'chatbot.png'
image_widget = widgets.Image.from_file(image_path, layout=widgets.Layout(width='250px'))
display(widgets.VBox([image_widget, button, output]))
